# Imports

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from survlimepy import SurvLimeExplainer
from survlimepy.load_datasets import RandomSurvivalData
from xgbse import XGBSEKaplanNeighbors
import pandas as pd
from pycox.models import DeepHitSingle, CoxPH
import torchtuples as tt

# Generate data

In [ ]:
# Generate data
n_points = 500
true_coef = [1, 1]
r = 1
center = [0, 0]
prob_event = 0.9
lambda_weibull = 10**(-6)
v_weibull = 2
n_features = len(true_coef)

rsd = RandomSurvivalData(
    center=center,
    radius=r,
    coefficients=true_coef,
    prob_event=prob_event,
    lambda_weibull=lambda_weibull,
    v_weibull=v_weibull,
    time_cap=None,
    random_seed=90,
)

# Train
X, time_to_event, delta = rsd.random_survival_data(num_points=n_points)
z = [(d, t) for d, t in zip(delta, time_to_event)]
y = np.array(z, dtype=[("delta", np.bool_), ("time_to_event", np.float32)])
total_row_train = X.shape[0]
print('total_row_train:', total_row_train)
unique_times = np.sort(np.unique(time_to_event))

In [ ]:
# Point to explain
x_new = np.array([0.1, 0.1])

# Cox

In [ ]:
# Fit a Cox model
cox = CoxPHSurvivalAnalysis()
cox.fit(X, y)
print(cox.coef_)

In [ ]:
# SurvLime for COX
explainer_cox = SurvLimeExplainer(
    training_features=X,
    training_events=[tp[0] for tp in y],
    training_times=[tp[1] for tp in y],
    model_output_times=cox.event_times_,
    sample_around_instance=True,
    random_state=10,
)

b_cox = explainer_cox.explain_instance(
    data_row=x_new,
    predict_fn=cox.predict_cumulative_hazard_function,
    num_samples=100,
    verbose=False,
)

explainer_cox.plot_weights()

# Random Survival Forest

In [ ]:
rsf = RandomSurvivalForest().fit(X, y)

In [ ]:
# SurvLime for RSF
explainer_rsf = SurvLimeExplainer(
    training_features=X,
    training_events=[tp[0] for tp in y],
    training_times=[tp[1] for tp in y],
    model_output_times=rsf.event_times_,
    sample_around_instance=True,
    random_state=10,
)

b_rsf = explainer_rsf.explain_instance(
    data_row=x_new,
    predict_fn=rsf.predict_cumulative_hazard_function,
    num_samples=1000,
    verbose=False,
)

explainer_rsf.plot_weights()

# xgbse

In [ ]:
# Transform the data in order to have the xgbse format
X_df = pd.DataFrame(X, columns = ['A', 'B'])

In [ ]:
xgbse = XGBSEKaplanNeighbors(n_neighbors=50)
xgbse.fit(X_df, y)

In [ ]:
explainer_xgbse = SurvLimeExplainer(
    training_features=X_df,
    training_events=[tp[0] for tp in y],
    training_times=[tp[1] for tp in y],
    model_output_times=xgbse.time_bins,
    sample_around_instance=True,
    random_state=10,
)

b_xgbse = explainer_xgbse.explain_instance(
    data_row=x_new,
    predict_fn=xgbse.predict,
    num_samples=1000,
    verbose=False
)

explainer_xgbse.plot_weights()

# DeepHit

In [ ]:
# Transform the data in order to have the DeepHit format
X_transformed = X.astype('float32')

In [ ]:
# Configure the net
in_features = X.shape[1]
num_nodes = [32, 32]
batch_norm = True
dropout = 0.1
output_bias = False
batch_size = 256
epochs = 512
get_target = lambda df: (df['duration'].values, df['event'].values)
verbose = True
num_durations = 50
labtrans = DeepHitSingle.label_transform(num_durations)
y_transformed = labtrans.fit_transform(time_to_event[:, 0], delta)

In [ ]:
net_deep_hit = tt.practical.MLPVanilla(
    in_features,
    num_nodes,
    labtrans.out_features,
    batch_norm,
    dropout,
    output_bias=output_bias
)
deep_hit = DeepHitSingle(net_deep_hit, tt.optim.Adam, alpha=0.2, sigma=0.1, duration_index=labtrans.cuts)
deep_hit.optimizer.set_lr(0.001)

# Train
log = deep_hit.fit(
    input=X_transformed,
    target=y_transformed,
    batch_size=batch_size,
    epochs=epochs,
    verbose=False
)

In [ ]:
explainer_deephit = SurvLimeExplainer(
    training_features=X_transformed,
    training_events=[tp[0] for tp in y],
    training_times=[tp[1] for tp in y],
    model_output_times=deep_hit.duration_index,
    sample_around_instance=True,
    random_state=10,
)

# Note that we use type_fn = "survival" since predict_fn is survival.
# Althought we use "survival", internally we transform everything to work with the cumulative hazard function.
# Therefore, the results must be interpret in the same way as if we were passing the cumulative hazard function.
b_deephit = explainer_deephit.explain_instance(
    data_row=x_new,
    predict_fn=deep_hit.predict_surv,
    type_fn = "survival",
    num_samples=1000,
    verbose=False,
)

explainer_deephit.plot_weights()

# DeepSurv
A per when this notebook is created, the version of `pycox` is 0.2.3. For DeepSuv model, the `predict_surv` return a mtriz of size $n \times m$, where $n$ is the number of individuals and $m$ is the number of unique times.

However, the function `predict_cumulative_hazards` returns a data frame of size $m \times n$. Therefore, if we want to use this function, we have to create a wrapper to transpose the output, since `SurvLIMEpy` needs the individuals in the rows. In this notebook, we show how to do that.

In [ ]:
y_df = pd.DataFrame(data={'duration': time_to_event[:, 0], 'event': delta})
y_deepsurv = get_target(y_df)

In [ ]:
net_deep_surv = tt.practical.MLPVanilla(in_features, num_nodes, 1, batch_norm,dropout, output_bias=output_bias)
deep_surv = CoxPH(net_deep_surv, tt.optim.Adam())
deep_surv.optimizer.set_lr(0.001)
log = deep_surv.fit(
    input=X_transformed,
    target=y_deepsurv,
    batch_size=batch_size,
    epochs=epochs,
    verbose=False
)

In [ ]:
deep_surv.compute_baseline_hazards()

In [ ]:
def create_chf(fun):
    def inner(X):
        Y = fun(X)
        return Y.T
    return inner

predict_chf = create_chf(deep_surv.predict_cumulative_hazards)

In [ ]:
explainer_deepsurv = SurvLimeExplainer(
    training_features=X_transformed,
    training_events=[tp[0] for tp in y],
    training_times=[tp[1] for tp in y],
    model_output_times=unique_times,
    sample_around_instance=True,
    random_state=10,
)

b_deepsurv = explainer_deepsurv.explain_instance(
    data_row=x_new,
    predict_fn=predict_chf,
    num_samples=1000,
    verbose=False,
)

explainer_deepsurv.plot_weights()